# Tarea para el Hogar 02

Esta Tarea para el Hogar 02 se entrega el final de la segunda clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 03, ya que se analizarán los resultados

##  1. Ensembles de Modelos

Vea el siguiente video [BBC - The Code - The Wisdom of the Crowd](https://www.youtube.com/watch?v=iOucwX7Z1HU)    ( 5 min)


Lea los siguientes artículos


*   [The Wisdom of Crowds (Vox Populi) by Francis Galton](https://www.all-about-psychology.com/the-wisdom-of-crowds.html)  (10 min)
*   [A Gentle Introduction to Ensemble Learning](https://machinelearningmastery.com/what-is-ensemble-learning/)  (10 min)





---



##  2.  Zero2Hero   primera parte
Se han lanzado los primeros fascículos coleccionables llamados "from Zero to Hero" que muy detalladamente, paso a paso enseñan todo lo necesario de R para entender los scripts oficiales de la asignatura.
Están en el repositorio oficial de la asignatura, carpeta  **src/zero2hero**



---



## 3.  Grid Search

Busque en internet el precido significado de los hiperparámetros de la librería **rpart**  que está implementando el algoritmo **CART**  Classification and Regression Trees  propuesto en el año 1984 por Leo Brieman:

*   cp
*   maxdepth
*   minsplit
*   minbucket

Entienda que valores es razonable tome cada hiperparámetro,  en particular profundice en el hiperparámetro  **cp**  y la posibilidad que tome valores negativos.  Es válido consultar a su amigo de *capacidades especiales*  ChatGPT


En las siguientes celdas a un notebook incompleto, un esqueleto de codigo brindado a modo de facilitarle la tarea de codeo y permitir que su valiosa cognición se concentre temas conceptuales de Ciencia de Datos

Modifiquelo agregando loops para que recorra TODOS los hiperparámetros de rpart  < cp, maxdepth, minsplit, minbucket >, y luego póngalo a correr. Recuerde cambiar por SU semilla
Tenga muy presente la granularidad que eligirá para cada hiperparámetro.

### Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



ln: failed to create symbolic link '/content/buckets/b1/labo1': File exists


limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657245,35.2,1454462,77.7,1326124,70.9
Vcells,1220272,9.4,8388608,64.0,1975127,15.1


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if (!require("primes")) install.packages("primes")
require("primes")

Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: primes



Aqui debe poner SU semiila primigenia

In [36]:
PARAM <- list()
# reemplazar por su primer semilla
PARAM$semilla_primigenia <- 765179
# PARAM$qsemillas <- 10

PARAM$training_pct <- 70L  # entre  1L y 99L

# elegir SU dataset comentando/ descomentando
PARAM$dataset_nom <- "/content/datasets/dataset_pequeno.csv"

In [87]:
# ---- per-machine config (set BEFORE loops) ----
# Laptop session:
#   Sys.setenv(CHUNK_K="2", CHUNK_ID="1", SOURCE="Laptop")
# Colab session:
#   Sys.setenv(CHUNK_K="2", CHUNK_ID="2", SOURCE="Google Colab")

chunk_k  <- as.integer(Sys.getenv("CHUNK_K",  unset = "2"))
chunk_id <- as.integer(Sys.getenv("CHUNK_ID", unset = "1"))

# generate shared seed list, then pick THIS machine's slice
PARAM$semilla_primigenia <- if (is.null(PARAM$semilla_primigenia)) 765179L else PARAM$semilla_primigenia
PARAM$qsemillas_total    <- if (is.null(PARAM$qsemillas_total))    1L    else PARAM$qsemillas_total

set.seed(PARAM$semilla_primigenia)
all_seeds <- sample.int(1e6, PARAM$qsemillas_total)
semillas  <- all_seeds[((seq_along(all_seeds) - 1L) %% chunk_k) + 1L == chunk_id]

# your skip-guard uses this:
PARAM$qsemillas <- length(semillas)



In [88]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}

In [90]:
# ==============================
# Parallel Monte Carlo runner
# - Runs ArbolEstimarGanancia once per seed in 'semillas'
# - Uses (almost) all cores on laptops/desktops; all cores on Colab
# ==============================
ArbolesMontecarlo <- function(semillas, param_basicos) {

  # Decide cores here (single source of truth)
  host <- tolower(Sys.getenv("HOSTNAME", ""))
  is_colab <- grepl("colab|google", host)

  n_detected <- parallel::detectCores()
  mc.cores   <- if (is_colab) n_detected else max(1L, n_detected - 1L)

  # Run one parallel call per seed (returns a list; caller can rbindlist)
  salida <- mcmapply(
    FUN       = ArbolEstimarGanancia,
    semillas,  # vector of seeds for THIS machine
    MoreArgs  = list(PARAM$training_pct, param_basicos, PARAM$dataset_nom),
    SIMPLIFY  = FALSE,
    mc.cores  = mc.cores
  )

  return(salida)
}


In [44]:
# carpeta de trabajo
# por fabor cambiar numero de experimento si se cambia el loop principal
setwd("/content/buckets/b1/exp")
experimento <- "HT2900"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [45]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [91]:
.one_run <- function(seed, training_pct, param_basicos, dataset_path) {
  ArbolEstimarGanancia(seed, training_pct, param_basicos)
}

.to_dt <- function(res, seed, extra=list()) {
  # Accept list/data.frame/atomic return types
  if (is.list(res) || is.data.frame(res)) {
    out <- as.data.table(res)
  } else {
    out <- data.table(result = as.numeric(res))
  }
  # annotate with key info
  for (nm in names(extra)) out[[nm]] <- extra[[nm]]
  out[, seed := seed][]
}

In [92]:

# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )


In [93]:
# genero la data.table donde van los resultados detallados del Grid Search
# un registro para cada combinacion de < semilla, parametros >

if(file.exists("gridsearch_detalle_chunk%d.txt")){
  tb_grid_search_detalle <- fread("gridsearch_detalle_chunk%d.txt")
}else{
  tb_grid_search_detalle <- data.table(
    semilla = integer(),
    cp = numeric(),
    maxdepth = integer(),
   minsplit = integer(),
    minbucket = integer(),
    ganancia_test = numeric()
  )
}

nrow( tb_grid_search_detalle )

[1] 0

Esta es la parte del código que usted debe expandir a TODOS los hiperparámetros de rpart,
<br>ya que actualmente apenas recorre  maxdepth y  minsplit  dejando fijos  cp=-0.5  y minbucket=5

In [ ]:
# ==============================
# Grid Search with Split-by-Seeds (multi-machine safe)
# Assumes:
#   - 'semillas' already contains THIS MACHINE's seed subset
#   - 'chunk_id' is defined (e.g., as.integer(Sys.getenv("CHUNK_ID", "1")))
#   - 'tb_grid_search_detalle' exists (data.table)
# ==============================

iter <- 0

for (vmax_depth in c(4, 6, 8, 10, 12, 14)) {
  for (vmin_split in c(1000, 800, 600, 400, 200, 100, 50, 20, 10)) {

    # progress
    iter <- iter + 1
    cat(iter, " ")
    flush.console()

    # keep your skip-guard logic intact
    if (iter * PARAM$qsemillas < nrow(tb_grid_search_detalle) + 1) next

    # params for this tree
    param_basicos <- list(
      "cp"        = -0.5,
      "maxdepth"  = vmax_depth,
      "minsplit"  = vmin_split,
      "minbucket" = 5
    )

    # run Monte Carlo for THIS machine's seeds
    res_list <- mcmapply(
      FUN       = .one_run,
      seed      = semillas,
      MoreArgs  = list(PARAM$training_pct, param_basicos, PARAM$dataset_nom), # Pass dataset path
      SIMPLIFY  = FALSE,
      mc.cores  = cores
    )

    # bind results and tag with hyperparams
    DT <- rbindlist(
      Map(.to_dt, res_list, semillas,
          MoreArgs = list(extra = list(
            maxdepth = vmax_depth,
            minsplit = vmin_split,
            cp       = param_basicos$cp
          ))),
      fill = TRUE
    )

  # --- tag (optional) ---
# Sys.setenv(SOURCE = "Laptop")   # or "Google Colab"
source_label <- Sys.getenv("SOURCE", unset = "Unknown")
data.table::set(DT, j = "source",   value = source_label)

# --- append to the in-memory master table ---
if (!exists("tb_grid_search_detalle") || !data.table::is.data.table(tb_grid_search_detalle)) {
  tb_grid_search_detalle <- data.table::data.table()
}
tb_grid_search_detalle <- data.table::rbindlist(
  list(tb_grid_search_detalle, DT),
  use.names = TRUE,
  fill = TRUE
)
  }
}

1  

In [94]:
# cantidad de registros de la tabla
nrow(tb_grid_search_detalle)

[1] 0

In [ ]:
# muestro la tabla
tb_grid_search_detalle

In [67]:
parts <- list.files(pattern = "^gridsearch_detalle_chunk\\d+\\.txt$", full.names = TRUE)
tb_merged <- data.table::rbindlist(lapply(parts, data.table::fread), fill = TRUE)
data.table::fwrite(tb_merged, "gridsearch_detalle_merged.txt", sep = "\t")


Warning message in data.table::fwrite(tb_merged, "gridsearch_detalle_merged.txt", :
“Input has no columns; creating an empty file at 'gridsearch_detalle_merged.txt' and exiting.”


In [ ]:
# muestro la tabla
tb_merged

In [12]:
# genero y grabo el resumen
tb_grid_search <- tb_merged[,
  list( "ganancia_mean" = mean(ganancia_test),
    "qty" = .N ),
  list( cp, maxdepth, minsplit, minbucket )
]


In [14]:
# ordeno descendente por ganancia
setorder( tb_grid_search, -ganancia_mean )


In [15]:
# veo los 10 mejores hiperparámetros
tb_grid_search[1:10]

cp,maxdepth,minsplit,minbucket,ganancia_mean,qty
<dbl>,<int>,<int>,<int>,<dbl>,<int>
-0.6,6,875,325,47940000,1
-0.6,6,850,325,47940000,1
-0.6,6,825,325,47940000,1
-0.6,6,800,325,47940000,1
-0.6,6,775,325,47940000,1
-0.6,6,750,325,47940000,1
-0.6,6,725,325,47940000,1
-0.6,6,700,325,47940000,1
-0.6,6,675,325,47940000,1


In [ ]:
# genero un id a la tabla
tb_grid_search[, id := .I ]

fwrite( tb_grid_search,
  file = "gridsearch.txt",
  sep = "\t"
)


# 4.  Análisis de resultados de Grid Search

La salida de la corrida anterior queda en ~/buckets/b1/exp/HT2900  que corresponde a su Google Drive
<br>HT significa Hyperparameter Tuning
<br>El Grid Search es un método de fuerza bruta de un altísimo costo computacional.
<br>Queremos ver si es posible crear un algoritmo de optimización de hiperparámetros que se ahorre recorrer ciertas porciones muy malas del espacio de búsqueda. Algo del estilo “cada vez que pruebo una combinación de hiperparámetros donde  cp > 1 , la ganancia es muy mala, con lo cual ni vale la pena perder el tiempo explorando en esa region”


<br>Levante el archivo de salida gridsearch.txt  a una planilla tipo Excel y analícelo detenidamente
<br>Ordene por ganancia_mean descendente
<br>
<br>El de mayor ganancia_mean  decimos que es el primero del ranking
En Zulip, correspondiente channel  #Tarea Hogar 02 , topic Analisis Grid Search   intente contestar estas preguntas:

* ¿Qué combinaciones de hiperparámetros poseen una ganancia muy buena?
* ¿Hay algun hiperparámetro que para cierto valor siempre genera una ganancia muy mala, a independientemente de lo que valgan los otros hiperparámetros ?
* ¿Que combinaciones de hiperparámetros es pésima y hubiera sido bueno ahorrarse esas corridas ?

( tiempo estimado 30 minutos, dificultad media )